In [11]:
#author: James Chan © 2018
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import st_utils as ut
import datetime as dt
%matplotlib inline

In [15]:
#We are concerned about tech sentiment, QQQ will be our stock (ETF) of interest.
sd = dt.datetime(2009,1,1)
ed = dt.datetime(2010,1,1)
dates = pd.date_range(sd, ed)
df = ut.get_data('QQQ.csv', dates) 


In [18]:
'''
Assumptions:
1. We assume we enter buy order at market close, hence using adjusted close. 
2. The news 
'''
df.dropna()
df['Daily Return'] = df['Adj Close'].pct_change()

In [19]:
#using FAANG not only because it's a cool buzzword, but they are also the tech market drivers
companies = ['facebook','apple','amazon','netflix','google']
companies = ['amazon']
start_date = '2018-07-25'
end_date = '2018-07-28'
news_source = 'wsj.com'
apikey = '23e7e5c41b6443f1bb556ec312275aff'
query_urls = []
for company in companies:
    url = 'https://newsapi.org/v2/everything?q={}&from={}&to={}&domains={}&apiKey={}'\
            .format(company, start_date, end_date, news_source, apikey)
    query_urls.append(url)


#data provided by https://newsapi.org <----these guys are awesome :-)!
for url in query_urls:
    response = requests.get(url)
    news = response.json()

In [ ]:
# pandas = 
#data preprocess:
#1. use description only, remove entries that don't actually have the company name mentioned in the description
#2. need to offset time by -4 hours to get ET time.  e.g., 1:00pm ET (summer) is shown as 17:00.
#3. bin the time to market open and market close per day.
for article in news['articles']:
    print(article['title'])
    print(article['description'])
    print()
    print(article['publishedAt'])

In [20]:
df

,Adj Close,Daily Return
2009-01-01,NaN,NaN
2009-01-02,28.152836,NaN
2009-01-03,NaN,NaN
2009-01-04,NaN,NaN
2009-01-05,28.143766,-0.000322
2009-01-06,28.425024,0.009994
2009-01-07,27.617548,-0.028407
2009-01-08,27.907877,0.010512
2009-01-09,27.281853,-0.022432
2009-01-10,NaN,NaN
